# Web Scraping and Web API

In [1]:
#Retrieving data from web pages or performing HTTP requests in Python.
#Helpful functions for analysing and extracting data from HTML and XML documents
#Suitable for data structures and data analysis
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Allocation URL
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
#Sends an HTTP GET request to the saved URL
data = requests.get(standings_url)

In [4]:
#analyse and search the HTML code of the website
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [5]:
#All URLs
team_urls = [f"https://fbref.com{l}" for l in links]

In [6]:
#HTTP GET request to the URL
#Index 0 = take the first URL from the teams_urls list
data = requests.get(team_urls[0])

In [7]:
#Extracting data (HTML tables) only tables scores and fixtures
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [8]:
#analyse HTML content and extract specific links (context: different competitors)
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [9]:
#HTTP GET request to the URL
data = requests.get(f"https://fbref.com{links[0]}")

In [10]:
#HTML tables to be extracted from the text content of the variable "data".
#Only tables with the heading Shooting
shooting = pd.read_html(data.text, match="Shooting")[0]

In [11]:
#Preview first pair of lines
shooting.head()

For Manchester City                                                     \
                 Date   Time              Comp                Round  Day   
0          2022-07-30  17:00  Community Shield  FA Community Shield  Sat   
1          2022-08-07  16:30    Premier League          Matchweek 1  Sun   
2          2022-08-13  15:00    Premier League          Matchweek 2  Sat   
3          2022-08-21  16:30    Premier League          Matchweek 3  Sun   
4          2022-08-27  15:00    Premier League          Matchweek 4  Sat   

                                         ... Standard               Expected  \
     Venue Result GF GA        Opponent  ...     Dist   FK PK PKatt       xG   
0  Neutral      L  1  3       Liverpool  ...      NaN  NaN  0     0      NaN   
1     Away      W  2  0        West Ham  ...     18.7  1.0  1     1      2.2   
2     Home      W  4  0     Bournemouth  ...     17.5  0.0  0     0      1.7   
3     Away      D  3  3   Newcastle Utd  ...     16.2  1.0  0     0      2.1   
4     Home      W  4  2  Crystal Palace  ...     14.1  0.0  0     0      2.2   

                            Unnamed: 25_level_0  
  npxG npxG/Sh G-xG np:G-xG        Match Report  
0  NaN     NaN  NaN     NaN        Match Report  
1  1.4    0.11 -0.2    -0.4        Match Report  
2  1.7    0.09  1.3     1.3        Match Report  
3  2.1    0.10  0.9     0.9        Match Report  
4  2.2    0.13  1.8     1.8        Match Report  

[5 rows x 26 columns]

In [12]:
#Top level of column heading is removed and the remaining one is set as heading
shooting.columns = shooting.columns.droplevel()

In [13]:
#Data fusion (merge) between the DataFrames "Matches" and "shooting", resulting in the table Date.
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [14]:
#Preview first few lines
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-07-30,17:00,Community Shield,FA Community Shield,Sat,Neutral,L,1.0,3.0,Liverpool,...,4-3-3,Craig Pawson,Match Report,NaN,14,8,NaN,NaN,0,0
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,...,4-3-3,Michael Oliver,Match Report,NaN,13,1,18.7,1.0,1,1
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,...,4-2-3-1,David Coote,Match Report,NaN,19,7,17.5,0.0,0,0
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,...,4-3-3,Jarred Gillett,Match Report,NaN,21,10,16.2,1.0,0,0
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,...,4-2-3-1,Darren England,Match Report,NaN,18,5,14.1,0.0,0,0


In [15]:
#Descending list with years from 2023 to 2020
years = list(range(2023, 2020, -1))
all_matches = []

In [16]:
#URL value
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats" 

In [17]:
#Call up and process data from different years
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [18]:
len(all_matches)

60

In [19]:
#Merging the data
match_df = pd.concat(all_matches)

In [20]:
#Convert column header to lower case
match_df.columns = [c.lower() for c in match_df.columns] 

In [21]:
#DataFrame
match_df 

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,...,Match Report,NaN,13.0,1.0,18.7,1.0,1.0,1.0,2023,Manchester City
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,...,Match Report,NaN,19.0,7.0,17.5,0.0,0.0,0.0,2023,Manchester City
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,...,Match Report,NaN,21.0,10.0,16.2,1.0,0.0,0.0,2023,Manchester City
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,...,Match Report,NaN,18.0,5.0,14.1,0.0,0.0,0.0,2023,Manchester City
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6.0,0.0,Nott'ham Forest,...,Match Report,NaN,17.0,9.0,14.8,0.0,0.0,0.0,2023,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0,4,Tottenham,...,Match Report,NaN,8.0,1.0,18.2,0.0,0.0,0.0,2021,Sheffield United
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0,2,Crystal Palace,...,Match Report,NaN,7.0,0.0,13.4,1.0,0.0,0.0,2021,Sheffield United
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1,0,Everton,...,Match Report,NaN,10.0,3.0,18.5,0.0,0.0,0.0,2021,Sheffield United
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0,1,Newcastle Utd,...,Match Report,NaN,11.0,1.0,18.3,1.0,0.0,0.0,2021,Sheffield United


In [22]:
#Export DataFrame as CSV file
match_df.to_csv("Football_Matches.csv")

# Storage of the data in MySQL

In [47]:
#install mysql connector
!pip  install mysql-connector-python

import mysql.connector

In [48]:
#Connect to the database
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password ="Technik7901$",
    database="football"
)

In [49]:
#Creating cursor
cursor = mydb.cursor()

In [ ]:
#Creating Database
#cursor.execute("CREATE DATABASE football")

In [52]:
#Creating and show Tables
#cursor.execute("CREATE TABLE matches (id INT AUTO_INCREMENT PRIMARY KEY, team VARCHAR(255), round VARCHAR(255), venue VARCHAR(255), time VARCHAR(255), result VARCHAR(255))")

In [53]:
#Adding data
insert_query = "INSERT INTO matches (team, round, venue, time, result) VALUES (%s, %s, %s, %s, %s)"
values = [
    ('Liverpool', '1', 'Home', '45', 'W'),
    ('Liverpool', '2', 'Home', '45', 'W'),
    ('Liverpool', '3', 'Home', '45', 'D'),
    ('Liverpool', '4', 'Home', '45', 'D'),
    ('Liverpool', '5', 'Home', '45', 'W'),
    ('Liverpool', '6', 'Home', '45', 'W'),
    ('Liverpool', '7', 'Home', '45', 'D'),
    ('Liverpool', '8', 'Home', '45', 'L'),
    ('Liverpool', '9', 'Home', '45', 'L'),
    ('Liverpool', '10', 'Home', '45', 'L'),
    ('Liverpool', '11', 'Home', '45', 'L'),
    ('Liverpool', '12', 'Home', '45', 'W'),
    ('Liverpool', '13', 'Home', '45', 'L'),
    ('Liverpool', '14', 'Home', '45', 'W'),

]

#Insert data in the data base
cursor.executemany(insert_query, values)

#Confirm changes
mydb.commit()

In [54]:
#Close connection
cursor.close()
mydb.close()
print(cursor.rowcount, "was inserted.")

14 was inserted.


### Retrieve all rows in the Matches table

In [55]:
#Connect to the database
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password ="Technik7901$",
    database="football"
)

#Creating cursor
cursor = mydb.cursor()

#SQL command to retrieve all rows from the table
select_query = "SELECT * FROM matches"


#Execute SQL command
cursor.execute(select_query)

#Retrieve all lines
rows = cursor.fetchall()

#Print lines
for row in rows:
    print(row)
    
#Close connection
cursor.close()
mydb.close()


(29, 'Liverpool', '1', 'Home', '45', 'W')
(30, 'Liverpool', '2', 'Home', '45', 'W')
(31, 'Liverpool', '3', 'Home', '45', 'D')
(32, 'Liverpool', '4', 'Home', '45', 'D')
(33, 'Liverpool', '5', 'Home', '45', 'W')
(34, 'Liverpool', '6', 'Home', '45', 'W')
(35, 'Liverpool', '7', 'Home', '45', 'D')
(36, 'Liverpool', '8', 'Home', '45', 'L')
(37, 'Liverpool', '9', 'Home', '45', 'L')
(38, 'Liverpool', '10', 'Home', '45', 'L')
(39, 'Liverpool', '11', 'Home', '45', 'L')
(40, 'Liverpool', '12', 'Home', '45', 'W')
(41, 'Liverpool', '13', 'Home', '45', 'L')
(42, 'Liverpool', '14', 'Home', '45', 'W')


### Result of the Liverpool team

In [66]:
#Connect to the database
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password ="Technik7901$",
    database="football"
)

#Creating cursor
cursor = mydb.cursor()

#SQL command for categorising and counting the results
select_query = """
    SELECT result, COUNT(*) AS count
    FROM matches
    WHERE result IN ('W', 'D', 'L')
    GROUP BY result
"""

#Execute SQL command
cursor.execute(select_query)

#Retrieve all lines
rows = cursor.fetchall()

# Ergebnisse drucken
for row in rows:
    rows = row[0]
    count = row[1]
    print(f"Kategorie {rows}: {count} Einträge")
    
# Verbindung schließen
cursor.close()
mydb.close()


Kategorie W: 6 Einträge
Kategorie D: 3 Einträge
Kategorie L: 5 Einträge
